In [4]:
import pandas as pd

df = pd.read_excel('http://projects.bobbelderbos.com/pcc/pandas/movies/movies.xlsx', sheet_name='data', header=7, usecols=[2,3] )

In [55]:
df = df[df.genres != '(no genres listed)']

In [56]:
# Create new Dataframe
def extract_values(row):
    genres = row['genres'].split('|')
    movie = row['movie']
    return [[genre, movie] for genre in genres]

data = df.apply(extract_values, axis=1)

In [57]:
df_new = pd.DataFrame.from_records([y for x in data for y in x], columns=['genres', 'movie'])

In [65]:
df_new.groupby('genres').count().sort_values('movie', ascending=False)

,movie
genres,
Drama,485
Comedy,302
Thriller,158
Action,156
Romance,146
Crime,124
Horror,92
Adventure,90
Documentary,82
